In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import glob
from memory_limits import duck_options
from datetime import datetime
from pathlib import Path


In [2]:
data_dir = Path("../data")

mlhd_path = data_dir / "mlhdv2"
musicbrainz_path = data_dir / "musicbrainz.db"
mlhd_ids_path = data_dir / "mlhd_ids.parquet"


conn = duckdb.connect(config=duck_options())
brainz_conn = duckdb.connect(musicbrainz_path, config=duck_options())

-1 -1 270210306048
-1 -1 270210306048


In [3]:
conn.execute(f"""create temp table mlhd_ids as 
                select * from read_parquet('{mlhd_ids_path}') 
                ORDER BY entity_uuid""")

In [4]:
conn.execute(f"attach '{musicbrainz_path}' as musicbrainz")

In [5]:
# map musicbrainz to mlhd_ids
conn.execute(f"""
    create temp table artists as
    SELECT 
        artist_uuid.id_num AS artist_id,
        mb_artist.gender AS gender,
        mb_artist.type AS type
    FROM musicbrainz.mb_artist as mb_artist
    RIGHT JOIN (SELECT * FROM mlhd_ids WHERE entity_type = 'artist_ids') 
                 AS artist_uuid
        ON artist_uuid.entity_uuid = mb_artist.artist_id
        """)

In [6]:
conn.execute("detach musicbrainz")

In [ ]:
# read training data
conn.execute(f"""create or replace temp mlhd_train as 
                select * from read_parquet('{mlhd_path}/*.parquet')
                order by timestamp asc
                limit (select CAST(count(*) * 0.8 as integer) from read_parquet('{mlhd_path}/*.parquet'))
""")

In [ ]:
# preprocessing on train


# exclude artists with fewer than 15 total listens across all users
del_artist_unpop = conn.execute("""
    DELETE FROM mlhd_train
    WHERE artist_id NOT IN (
        SELECT artist_id
        FROM mlhd_train
        GROUP BY artist_id
        HAVING COUNT(*) >= 15
    );
""")
deleted_rows = del_artist_unpop.fetchall()[0][0]
print(f"Number rows deleted for artists with <15 total listens: {deleted_rows}")

# exclude users with fewer than 15 distinct artists in their data
del_user_artist = conn.execute("""
    DELETE FROM mlhd_train
    WHERE user_id NOT IN (
        SELECT user_id, artist_id
        FROM mlhd_train,
        GROUP BY user_id
        HAVING COUNT(DISTINCT artist_id) >= 15
    );
""")

deleted_rows = del_user_artist.fetchall()[0][0]
print(f"Number of rows deleted for users with <15 artists : {deleted_rows}")

In [1]:
conn.close()

NameError: name 'conn' is not defined